In [1]:
from requests_html import HTMLSession
session = HTMLSession()

In [2]:
url = 'https://vi.wikipedia.org/wiki/Vua_Việt_Nam'
r = session.get(url)

# tìm trên web các phần của table(bảng)
tags = r.html.find('h2, h3, table.wikitable')

all_monarchs_data = []
current_header_name = "Không rõ triều đại" # Biến lưu tên tiêu đề gần nhất

# 2. Lặp qua danh sách các thẻ đã tìm được
for tag in tags:
    
    # Nếu thẻ là một tiêu đề, cập nhật lại tên triều đại gần nhất
    if tag.tag in ['h2', 'h3']:
        headline = tag.find('span.mw-headline', first=True)
        if headline:
            current_header_name = headline.text
        else:
            # Nếu không, lấy trực tiếp text từ thẻ h2/h3
            # Dùng .strip() để xóa các khoảng trắng thừa
            current_header_name = tag.text.strip()
            # Loại bỏ phần [sửa | sửa mã nguồn] nếu có
            if '[sửa | sửa mã nguồn]' in current_header_name:
                current_header_name = current_header_name.split('[sửa | sửa mã nguồn]')[0].strip()
    
    # Nếu thẻ là một cái bảng, tiến hành xử lý
    elif tag.tag == 'table':
        
        # Ưu tiên 1: Tìm caption bên trong bảng
        caption_tag = tag.find('caption', first=True)
        
        # Quyết định tên triều đại cuối cùng
        # Nếu có caption thì dùng, nếu không thì dùng tên tiêu đề đã lưu
        final_dynasty_name = caption_tag.text if caption_tag else current_header_name
        
        # Bóc tách dữ liệu từ các hàng trong bảng
        rows = tag.find('tr')[1:] # Bỏ qua hàng tiêu đề
        for row in rows:
            cells = row.find('td')
            # Cần kiểm tra số lượng ô để tránh lỗi với các bảng cấu trúc lạ
            if len(cells) > 5:
                ten_huy = cells[4].text
                nien_hieu = cells[5].text
                
                all_monarchs_data.append({
                    "Triều đại": final_dynasty_name,
                    "Tên húy": ten_huy,
                    "Niên hiệu": nien_hieu
                })

# In kết quả cuối cùng để kiểm tra
for monarch in all_monarchs_data:
    print(monarch)

{'Triều đại': '', 'Tên húy': 'không có', 'Niên hiệu': 'Khúc Thừa Dụ[20]\n(曲承裕)'}
{'Triều đại': '', 'Tên húy': 'không có', 'Niên hiệu': 'Khúc Hạo,[20]\n(曲顥)\nKhúc Thừa Hạo[21]'}
{'Triều đại': '', 'Tên húy': 'không có', 'Niên hiệu': 'Khúc Thừa Mỹ[21]\n(曲承美)\nKhúc Toàn Mỹ'}
{'Triều đại': 'Họ Dương (931–937)', 'Tên húy': 'không có', 'Niên hiệu': 'Dương Đình Nghệ,[20]\n(楊廷藝)\nDương Diên Nghệ[23]\n(楊筵藝)'}
{'Triều đại': '', 'Tên húy': 'không có', 'Niên hiệu': 'Kiều Công Tiễn,[21]\n(矯公羨)\nKiểu Công Tiễn,[20]\nKiểu Công Hạo,[23]'}
{'Triều đại': '', 'Tên húy': 'Thuận Thiên (1010–1028)[30]', 'Niên hiệu': 'Lý Công Uẩn[30]'}
{'Triều đại': '', 'Tên húy': 'Thiên Thành (1028–1033),[30]\nThông Thụy (1034–1038),[30]\nCàn Phù Hữu Đạo (1039–1041),[30]\nMinh Đạo (1042–1043),[30]\nThiên Cảm Thánh Vũ (1044–1048),[30]\nSùng Hưng Đại Bảo (1049–1054).[30]', 'Niên hiệu': 'Lý Phật Mã,[30]\nLý Đức Chính[30]'}
{'Triều đại': '', 'Tên húy': 'Long Thụy Thái Bình (1054–1058),[31] Chương Thánh Gia Khánh (1059–1065),[31]